In [1]:
import polars as pl

df = pl.scan_ndjson("../data/reddit/comments/RC_2015-01", infer_schema_length=10000)

# parsing
df = df.with_columns(
    pl.col("created_utc")
    .str.to_integer()
    .mul(1000)
    .cast(pl.Datetime(time_unit="ms"))
    .dt.replace_time_zone("UTC")
)

In [19]:
import polars as pl

df_submissions = pl.read_ndjson(
    "../data/reddit/submissions/RS_2015-01",
    ignore_errors=True,
    infer_schema_length=None,
)

In [23]:
df.tail().collect()

score_hidden,name,link_id,body,downs,created_utc,score,author,distinguished,id,archived,parent_id,subreddit,author_flair_css_class,author_flair_text,gilded,retrieved_on,ups,controversiality,subreddit_id,edited
bool,str,str,str,i64,"datetime[ms, UTC]",i64,str,str,str,bool,str,str,str,str,i64,i64,i64,i64,str,i64
false,"""t1_co77gzp""","""t3_2ubiys""","""Shhh, let's not turn this murd…",0,2015-01-31 23:59:59 UTC,1,"""Dymethyltryptamine""",null,"""co77gzp""",false,"""t1_co7350n""","""AdviceAnimals""",null,null,0,1424281770,1,0,"""t5_2s7tt""",0
false,"""t1_co77gzq""","""t3_2ucs3e""","""I use the lee filter system. …",0,2015-01-31 23:59:59 UTC,2,"""docfluty""",null,"""co77gzq""",false,"""t3_2ucs3e""","""photography""","""facebook""","""www.facebook.com/docfluty""",0,1424281770,2,0,"""t5_2qh2a""",0
false,"""t1_co77gzr""","""t3_2uayw8""","""Because it's set to flash by d…",0,2015-01-31 23:59:59 UTC,1,"""2kewl4skoool""",null,"""co77gzr""",false,"""t1_co742y8""","""PS4""","""ps""","""SlamMonsterT""",0,1424281770,1,0,"""t5_2rrlp""",0
false,"""t1_co77gzs""","""t3_2ucst9""","""looks good """,0,2015-01-31 23:59:59 UTC,1,"""trappinnorthtx""",null,"""co77gzs""",false,"""t3_2ucst9""","""supremeclothing""",null,null,0,1424281770,1,0,"""t5_2tual""",0
false,"""t1_co77gzt""","""t3_2uazsm""","""You can already shoot through …",0,2015-01-31 23:59:59 UTC,3,"""blolfighter""",null,"""co77gzt""",false,"""t1_co6zqmw""","""Warframe""","""valkyr-bastet""","""RRRAURGH!""",0,1424281770,3,0,"""t5_2urg0""",0


In [24]:
df_submissions.filter(pl.col("name").eq("t3_2ubiys"))

imp_pixel,promoted,score,quarantine,created,over_18,ups,name,distinguished,downs,secure_media,author,author_flair_css_class,mobile_ad_url,retrieved_on,permalink,secure_media_embed,hide_score,archived,author_flair_text,thumbnail,stickied,link_flair_css_class,is_self,num_comments,href_url,disable_comments,id,gilded,adserver_imp_pixel,media,url,link_flair_text,title,third_party_tracking,third_party_tracking_2,selftext,created_utc,saved,adserver_click_url,media_embed,domain,edited,from_kind,from_id,from,subreddit,subreddit_id,post_hint,preview
null,bool,i64,bool,i64,bool,i64,str,str,i64,struct[3],str,str,str,i64,str,struct[4],bool,bool,str,str,bool,str,bool,i64,str,bool,str,i64,null,struct[3],str,str,str,null,null,str,str,bool,null,struct[4],str,i64,null,null,null,str,str,str,struct[1]
null,null,4215,false,1422725108,false,4215,"""t3_2ubiys""",null,0,null,"""allisaurusrexxx""",null,null,1440934467,"""/r/AdviceAnimals/comments/2ubi…","{null,null,null,null}",false,true,null,"""http://b.thumbs.redditmedia.co…",false,null,false,363,null,null,"""2ubiys""",0,null,null,"""http://imgur.com/QRqkUEy""",null,"""My roommate has been chatting …",null,null,"""""","""1422725108""",false,null,"{null,null,null,null}","""imgur.com""",0,null,null,null,"""AdviceAnimals""","""t5_2s7tt""",null,null


In [ ]:
CHAR_PER_TOKEN = 4

with_tokens = df.with_columns(
    token_count_approx=pl.col("body").str.len_chars() / CHAR_PER_TOKEN
)

with_tokens.collect().write_ndjson("../data/reddit/comments/RC_2015-01_parsed.ndjson")